In [22]:
csvfile = r'C:\Users\roumba\Documents\Software\deep-events\Videos\21.09_mtstaygold_cos7_iSIM_1\mtStayGold__1_points.csv'
imagefile= r'C:\Users\roumba\Documents\Software\deep-events\Videos\21.09_mtstaygold_cos7_iSIM_1\mtStayGold__1_MMStack_Pos0.ome.tif'

from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.io
import plotly.express as px
from scipy import ndimage as ndi
import tensorflow
import imageio
from scipy import signal
import tensorflow_probability as tfp
import tensorflow as tf
from tqdm import tqdm  
import myfunctions
from myfunctions import event_separation , image_crop_save_gauss , get_gaussian , image_crop_save

img = Image.open(imagefile)
datacsv = pd.read_csv(csvfile)
sigma_trial=(8,8)
size_trial=(2048,2048)


In [23]:
points_of_interest= np.zeros((100, 2048, 2048))

for row_number in tqdm(range(0, len(datacsv))):
    framenumber_in_row= int(datacsv.loc[row_number, 'axis-0'])
    fission_ycoord = int(datacsv.loc[row_number, 'axis-1'])
    fission_xcoord = int(datacsv.loc[row_number, 'axis-2'])
    fission_coords=(fission_ycoord,fission_xcoord)
    gaussian_points=get_gaussian(fission_coords,sigma_trial,size_trial)                                     #gets gaussian points at a single frame
    gaussian_points = gaussian_points.numpy()                                                               #convers tensor into numpy array
    gaussian_points = gaussian_points/np.max(gaussian_points)                                               #divides by the max 
    gaussian_points[gaussian_points < 0.1] = 0                                                              #sets background to zero
    gaussian_points = gaussian_points/np.max(gaussian_points)                                               #divides by max again
    points_of_interest[framenumber_in_row] = points_of_interest[framenumber_in_row] + gaussian_points       #adds the gaussian intensity in the empty file 


   
imageio.mimwrite('points1isim.tiff', (points_of_interest*254).astype(np.uint8))

100%|██████████| 136/136 [00:29<00:00,  4.63it/s]


In [24]:
list1=event_separation(datacsv)
l=len(list1)
image_crop_save(l,list1, datacsv, img)

c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\tifffile\tifffile.py:2001: UserWarning: <tifffile.TiffWriter 'image_7.tiff'> writing zero-size array to nonconformant TIFF
  warnings.warn(


In [25]:
gauss=r'C:\Users\roumba\Documents\Software\deep-events\points1isim.tiff'
gauss_image=Image.open(gauss)
image_crop_save_gauss(l,list1, datacsv, gauss_image)

c:\Users\roumba\Documents\Software\deep-events\.env\lib\site-packages\tifffile\tifffile.py:2001: UserWarning: <tifffile.TiffWriter 'image_7gauss.tiff'> writing zero-size array to nonconformant TIFF
  warnings.warn(
